In [1]:
import numpy as np
import pandas as pd

### $\int_{-\inf}^{z} e^{\frac{-u^2}{2}}du = \frac{1-0.05}{2}*\sqrt{2*π}$

In [2]:
z_gamma_empty = 0.062706777

### Завдання 1

Выборочные блоки -- естественные разбиения оси $Ox$ на интервалы $ (X_{i-1},X_i],\ i=1,...,n+1\ (X_{(0)} = -\inf, X_{(n+1)} = \inf) $

Пусть $\xi_r = \xi_r(n,m) \larr$ число этих блоков, каждый из которых содержит ровно $r$ элементов второй выборки $Y=(Y_1,...,Y_m), r = 0,1,...,m$. Тогда в качестве тестовой статистики можно взять произвольную линейную комбинацию $\sum_r c_r \xi_r$. В часности, $\xi_0$ - число блоков, не содержащих ни одного элемента второй выборки, -- порождает критерий пустых блоков. 

In [3]:
# example 1
X = np.random.exponential(1, 10)
Y = np.random.exponential(1, 20)
intervals = []
for i in range(1, len(X + 1)):
    intervals.append([X[i - 1], X[i]])

res = np.zeros(len(intervals))
for i, x in enumerate(intervals):
    for item in Y:
        if intervals[i][0] <= item <= intervals[i][1]:
            print(f"{intervals[i][0]} <= {item} <= {intervals[i][1]}")
            res[i] += 1
    if res[i] != 0:
        print(f"{i} :: {res[i]}")
        print("+++++++++++++++++++++++++")

print()
print()
print(res)
print("+" * 20)
print(np.count_nonzero(res == 0))


0.08415206501534911 <= 0.16439365654301535 <= 0.19041034459900838
1 :: 1.0
+++++++++++++++++++++++++
0.19041034459900838 <= 0.2366463178588818 <= 0.8677577335026762
0.19041034459900838 <= 0.3318890941243998 <= 0.8677577335026762
0.19041034459900838 <= 0.7645044587574106 <= 0.8677577335026762
0.19041034459900838 <= 0.38334787239998236 <= 0.8677577335026762
0.19041034459900838 <= 0.26481454756429684 <= 0.8677577335026762
0.19041034459900838 <= 0.3522375236741401 <= 0.8677577335026762
0.19041034459900838 <= 0.3594595347965496 <= 0.8677577335026762
0.19041034459900838 <= 0.37974184434148234 <= 0.8677577335026762
2 :: 8.0
+++++++++++++++++++++++++
0.8677577335026762 <= 0.876500139145764 <= 1.6784198315476555
0.8677577335026762 <= 0.8766071373379802 <= 1.6784198315476555
0.8677577335026762 <= 1.5537822548311275 <= 1.6784198315476555
0.8677577335026762 <= 1.6234574325623725 <= 1.6784198315476555
0.8677577335026762 <= 1.562072727573546 <= 1.6784198315476555
0.8677577335026762 <= 0.893969368192

In [4]:
NM = [[500, 1000], [5000, 10000]]  #, [50000, 100000]]

xi_empty = []
crit_empty = []
RES_empty = []
for nm in NM:

    n = nm[0]
    m = nm[1]
    ro = m / n

    X = np.random.exponential(1, n)
    Y = np.random.exponential(1, m)
    intervals = []
    for i in range(1, len(X + 1)):
        intervals.append([X[i - 1], X[i]])

    res = np.zeros(len(intervals))
    for i, x in enumerate(intervals):
        for item in Y:
            if intervals[i][0] <= item <= intervals[i][1]: res[i] += 1

    xi_empty_temp = np.count_nonzero(res == 0)
    xi_empty.append(xi_empty_temp)

    crit = n / (1 + ro) + np.math.sqrt(n) * ro * z_gamma_empty / (1 + ro) ** (3 / 2)
    crit_empty.append(crit)
    if xi_empty_temp < crit:
        RES_empty.append("H0")
    else:
        RES_empty.append("False")

data_empty = {"xi_empty (n, m)": xi_empty, f"crit": crit_empty, "Result": RES_empty}
df_empty = pd.DataFrame.from_dict(data_empty, orient='index', columns=[f"n::{item[0]}; m::{item[1]}" for item in NM])


In [5]:
df_empty

,n::500; m::1000,n::5000; m::10000
"xi_empty (n, m)",244,2497
crit,167.206361,1668.373329
Result,False,False


## Завдання 2

### Ранжування

In [6]:
rank = lambda data: pd.Series(data).rank(method="min").to_list()
gamma = 0.05
z_gamma = 0.0627

### Критерій перевірки гіпотези $H_0$


#### $\rho_n(\bar X, \bar Y) = \frac {\sum^n_{i=1} (R_i  - \bar R) (S_i - \bar S)}  {[ \sum^n_{i=1} (R_i  - \bar R)^2 \sum^m_{i=1}(S_i - \bar S)^2 ]^{\frac{1}{2}}}$

##### $\bar R = \frac {1}{n} \sum_{i=1}^n \frac{n+1}{2}$
##### $\sum^n_{i=1} (R_i  - \bar R)^2 = \frac {n(n^2-1)}{12}$


### критей Спірмана

In [7]:
def SpearMan(V):
    n = len(V)
    return 1 - 6 * np.sum(pow(V[:, 0] - V[:, 1], 2)) / (n * (pow(n, 2) - 1))
    ...

In [8]:
X = [5, 2, 1, 7, 8, 4, 6, 3]
Y = [4, 3, 2, 1, 7, 6, 5, 8]

n = 8
print(n)

res = np.array([X, Y])
res = np.transpose(res)
res = res[res[:, 0].argsort()]

test = abs(SpearMan(res))
print(test)

8
0.16666666666666663


In [9]:
N_arr = [500, 5000, 50000]
z_gamma_spearman = [z_gamma / np.math.sqrt(n) for n in N_arr]

RHO = []
Hyp_Spearman = []
for i, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi * data_nu)

    res = np.array([X, Y])
    res = np.transpose(res)
    res = res[res[:, 0].argsort()]

    test = abs(SpearMan(res))
    RHO.append(test)
    if test < z_gamma_spearman[i]:
        Hyp_Spearman.append("H0")
    else:
        Hyp_Spearman.append("False")

data_Spearman = {"Spearman statistic": RHO, f"z_gamma": z_gamma_spearman, "Result": Hyp_Spearman}
df_Spearman = pd.DataFrame.from_dict(data_Spearman, orient='index', columns=N_arr)

df_Spearman

,500,5000,50000
Spearman statistic,0.034411,0.004637,0.004837
z_gamma,0.002804,0.000887,0.00028
Result,False,False,False


### критерій Кендала

In [24]:
N_arr = [500, 5000, 50000]

z_gamma_kendal = [z_gamma / np.math.sqrt(n) / 3 * 2 for n in N_arr]
tau = []
Hyp_Kendal = []
for tmp, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi * data_nu)

    res = np.array([X, Y], np.int64)
    res = np.transpose(res)
    res = res[res[:, 0].argsort()]
    P_q = 0
    for i in res[1:-1, 0]:
        if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

    P_p = n * (n - 1) / 2 - P_q
    res = 4 * P_p / (n * (n - 1)) - 1
    tau.append(res)
    if res < z_gamma_kendal[tmp]:
        Hyp_Kendal.append("H0")
    else:
        Hyp_Kendal.append("false")

data_Kendal = {"Kendal statistic": tau, f"z_gamma": z_gamma_kendal, "Result": Hyp_Kendal}
df_Kendal = pd.DataFrame.from_dict(data_Kendal, orient='index', columns=N_arr)

df_Kendal

,500,5000,50000
Kendal statistic,0.392834,0.383934,0.3887
z_gamma,0.001869,0.000591,0.000187
Result,false,false,false


In [11]:
data_xi = [100, 118, 112, 97, 99, 103, 102, 132, 122, 121, 115, 117, 109, 111]
data_nu = [154, 123, 120, 213, 200, 187, 155, 100, 114, 115, 107, 176, 143, 111]
n = len(data_xi)
X = rank(data_xi)
Y = rank(data_nu)

res = np.array([X, Y], np.int64)
res = np.transpose(res)
res = res[res[:, 0].argsort()]
P_q = 0
for i in res[1:-1, 0]:
    if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

P_p = n * (n - 1) / 2 - P_q
print(P_q, P_p, 4 * P_q / (n * (n - 1)) - 1)


15 76.0 -0.6703296703296704


### Завдання 3
#### Критерій інверсій

У даномоу методі будемо перевіряти гіпотезу випадковості (повного хаосу), що означає:
$$H_0: F_{\overline{x}}(u_1, \ldots, u_n) = F_{\xi}(u_1) \cdot F_{\xi}(u_2) \cdot \ldots \cdot F_{\xi}(u_n)$$

**Алгоритм обчислення:**
1. Згенерувати $n$ спостережень
2. Обчислюємо "хаос" $k$ у варіаційному ряді,
3. Обчислюємо критичну область $\dfrac{z_{\gamma} n \sqrt{n}}{6}$
4. Рахуємо статистику $S_n(\overline{X}) = \Big| k - \dfrac{n(n-1)}{4}  \Big|$
5. Порівнюємо $ \Big| k - \dfrac{n(n-1)}{4}  \Big| \bigvee  \dfrac{z_{\gamma} n \sqrt{n}}{6}$
як $<$, приймаємо гіпотезу $H_0$, інакше -- ні

In [28]:
N_arr = [500, 5000]  #, 50000]
z_gamma_inv = [z_gamma / 6 * n ** (3 / 2) for n in N_arr]
LAMBDA = []
RES_inv = []

for tmp, n in enumerate(N_arr):
    X = np.random.normal(-1, 1, n)
    k = 0

    for x in range(1, len(X) - 1):
        for j in range(x, len(X)):
            i = j - 1
            if (X[j] < X[i]) and (i < j): k += 1

    print(f"k_{tmp}: {k}")

    statistic = (abs(k - n * (n - 1) / 4))
    LAMBDA.append(statistic)
    if statistic < z_gamma_inv[tmp]:
        RES_inv.append("H0")
    else:
        RES_inv.append("false")

data_inv = {"LAMBDA": LAMBDA, f"z_gamma": z_gamma_inv, "Result": RES_inv}
df_inv = pd.DataFrame.from_dict(data_inv, orient='index', columns=N_arr)

df_inv

k_0: 62839
k_1: 6342518


,500,5000
LAMBDA,464.0,93768.0
z_gamma,116.834552,3694.632932
Result,false,false
